# Wav2Vec - English Single

In [1]:
import torch
import librosa
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import numpy as np
model_name = "skpawar1305/wav2vec2-base-finetuned-digits"
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)


In [2]:
import os
PATH = os.getcwd()

In [3]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import sounddevice as sd

test_file = PATH+"/12345_68772.wav"
print(test_file)
print(os.path.exists(test_file))
def get_nparray(audiosegment: AudioSegment):
    samples = audiosegment.get_array_of_samples()
    samples_float = librosa.util.buf_to_float(samples,n_bytes=4, dtype=np.float32)
    if audiosegment.channels==2:
        sample_left= np.copy(samples_float[::2])
        sample_right= np.copy(samples_float[1::2])
        sample_all = np.array([sample_left,sample_right])
    else:
        sample_all = samples_float

    return sample_all


data = AudioSegment.from_wav(test_file)
print(f"db = {data.dBFS}")
sd.play(get_nparray(data), samplerate=16000)

D:\pycharm\digital_signal\speech_signal_processing\ThirdPartyRepos\Wav2Vec/12345_68772.wav
True
db = -21.15406995820191


In [4]:
from tqdm import tqdm
chunks = split_on_silence(data, min_silence_len=20, keep_silence=40, silence_thresh=-25)
print(len(chunks))
for _c in tqdm(chunks):
    sd.play(get_nparray(_c), samplerate=16000)
    sd.wait()

# sd.play(get_nparray(data))


9


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:06<00:00,  1.43it/s]


In [5]:
def inference(chunk):
    inputs = feature_extractor(chunk, sampling_rate=16000, padding=True, return_tensors="pt")

    logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    labels = [model.config.id2label[_id] for _id in predicted_ids.tolist()]
    if not len(labels):
        print("dead")
    return labels[0]

for _c in chunks:
    print(inference(get_nparray(_c)))

one
_unknown_
_unknown_
two
_unknown_
_unknown_
three
four
five


# Wav2Vec - Chinese Sequence


In [ ]:
from huggingsound import SpeechRecognitionModel
from pprint import pprint
import os

model = SpeechRecognitionModel("wbbbbb/wav2vec2-large-chinese-zh-cn", device="cuda")

In [ ]:
dataset = './zh_seq'
dataset = [os.path.join(dataset, f) for f in os.listdir(dataset)]
eval_files = dataset.copy()
tmp = []
for d in dataset:
    label = os.path.splitext(os.path.split(d)[1])[0].split('_')[0]
    tmp.append({"path": d, "transcription": label})
dataset = tmp
del tmp

In [ ]:
pprint(model.transcribe(dataset))

In [ ]:
model.finetune("train_output", dataset)